# TextMate syntax grammars in JupyterLab

In [ ]:
from pathlib import Path
import json

For demonstration purposes, we need a new MIME type.

In [ ]:
tmlanguage_mime = "application/vnd.jupyter.tmlanguage.v1+json"

Maybe go get an example language, in this case for [Stan](https://github.com/stan-dev).

In [ ]:
stan_json = Path("language-stan.json")
if not stan_json.exists():
    !wget https://raw.githubusercontent.com/jrnold/atom-language-stan/master/grammars/language-stan.json

We'll also want a largish example of a stan file... luckily there's an extensive [example repo](https://github.com/stan-dev/example-models).

In [ ]:
stan_examples = Path("example-models")
if not stan_examples.exists():
    !git clone https://github.com/stan-dev/example-models

The [tmlanguage spec](https://github.com/martinring/tmlanguage/blob/master/tmlanguage.json) apparently doesn't provide MIME types or file extensions, so we have to backfill.

In [ ]:
display({
    tmlanguage_mime: json.loads(stan_json.read_text())
}, metadata=dict(
    mode="Stan",
    name="Stan",
    ext=["stan"],
    mime="application/x-stan",
    options={"scopeTranslations": {
        "entity.name.type.stan": "header",
    }}
), raw=True)

If that worked, we should be able to start using `stan` as a type, and open [`.stan` files](./example-models/bugs_examples/vol1/stacks/stacks_b_dexp.stan) with nice syntax.
```stan
#DOC See \url{http://www.openbugs.net/Examples/Blockers.html}.
data {
  int<lower=0> N; 
  int<lower=0> nt[N]; 
  int<lower=0> rt[N]; 
  int<lower=0> nc[N]; 
  int<lower=0> rc[N]; 
} 
parameters {
  real d; 
  real<lower=0> sigmasq_delta; 
  vector[N] mu;
  vector[N] delta;
  real delta_new;
} 
transformed parameters {
  real<lower=0> sigma_delta; 
  sigma_delta <- sqrt(sigmasq_delta); 
} 
model {
  rt ~ binomial_logit(nt, mu + delta);
  rc ~ binomial_logit(nc, mu);
  delta  ~ student_t(4, d, sigma_delta); 
  mu ~ normal(0, sqrt(1E5));
  d ~ normal(0, 1E3); 
  sigmasq_delta ~ inv_gamma(1E-3, 1E-3); 
  // FIXME: sample in generated quantities in later version
  delta_new ~ student_t(4, d, sigma_delta); 
}
```